In [1]:
'''
Jan 12, 2017
CPM
NBA_shot_logs

https://www.kaggle.com/socathie/kickstarter-project-statistics
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
live = pd.read_csv('input/live.csv',index_col=0)
back = pd.read_csv('input/most_backed.csv',index_col=0)

In [8]:
live.head()

,amt.pledged,blurb,by,country,currency,end.time,location,percentage.funded,state,title,type,url
0,15823.0,"\n'Catalysts, Explorers & Secret Keepers: Wome...",Museum of Science Fiction,US,usd,2016-11-01T23:59:00-04:00,"Washington, DC",186,DC,"Catalysts, Explorers & Secret Keepers: Women o...",Town,/projects/1608905146/catalysts-explorers-and-s...
1,6859.0,\nA unique handmade picture book for kids & ar...,"Tyrone Wells & Broken Eagle, LLC",US,usd,2016-11-25T01:13:33-05:00,"Portland, OR",8,OR,The Whatamagump (a hand-crafted story picture ...,Town,/projects/thewhatamagump/the-whatamagump-a-han...
2,17906.0,\nA horror comedy about a repairman who was in...,Tessa Stone,US,usd,2016-11-23T23:00:00-05:00,"Los Angeles, CA",102,CA,Not Drunk Enough Volume 1!,Town,/projects/1890925998/not-drunk-enough-volume-1...
3,67081.0,\nThe Johnny Wander autobio omnibus you've all...,Johnny Wander,US,usd,2016-11-01T23:50:00-04:00,"Brooklyn, NY",191,NY,Our Cats Are More Famous Than Us: A Johnny Wan...,County,/projects/746734715/our-cats-are-more-famous-t...
4,32772.0,\nThe vision for this project is the establish...,Beau's All Natural Brewing Company,RW,cad,2016-11-18T23:05:48-05:00,"Kigali, Rwanda",34,Kigali Province,The Rwanda Craft Brewery Project,Town,/projects/beaus/the-rwanda-craft-brewery-proje...


In [15]:
live.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 12 columns):
amt.pledged          4000 non-null float64
blurb                4000 non-null object
by                   4000 non-null object
country              3999 non-null object
currency             4000 non-null object
end.time             4000 non-null object
location             4000 non-null object
percentage.funded    4000 non-null int64
state                4000 non-null object
title                4000 non-null object
type                 4000 non-null object
url                  4000 non-null object
dtypes: float64(1), int64(1), object(10)
memory usage: 406.2+ KB


In [16]:
live.isnull().any()

amt.pledged          False
blurb                False
by                   False
country               True
currency             False
end.time             False
location             False
percentage.funded    False
state                False
title                False
type                 False
url                  False
dtype: bool

In [11]:
live.describe()

,amt.pledged,percentage.funded
count,4.000000e+03,4000.000000
mean,1.023127e+04,97.164750
std,5.199757e+04,677.028552
min,0.000000e+00,0.000000
25%,1.000000e+02,1.000000
50%,1.035500e+03,18.000000
75%,5.450000e+03,74.000000
max,1.827576e+06,27997.000000


In [12]:
back.head()

,amt.pledged,blurb,by,category,currency,goal,location,num.backers,num.backers.tier,pledge.tier,title,url
0,8782571.0,\nThis is a card game for people who are into ...,Elan Lee,Tabletop Games,usd,10000.0,"Los Angeles, CA",219382,"[15505, 202934, 200, 5]","[20.0, 35.0, 100.0, 500.0]",Exploding Kittens,/projects/elanlee/exploding-kittens
1,6465690.0,"\nAn unusually addicting, high-quality desk to...",Matthew and Mark McLachlan,Product Design,usd,15000.0,"Denver, CO",154926,"[788, 250, 43073, 21796, 41727, 21627, 12215, ...","[1.0, 14.0, 19.0, 19.0, 35.0, 35.0, 79.0, 79.0...",Fidget Cube: A Vinyl Desk Toy,/projects/antsylabs/fidget-cube-a-vinyl-desk-toy
2,5408916.0,\nBring Reading Rainbow’s library of interacti...,LeVar Burton & Reading Rainbow,Web,usd,1000000.0,"Los Angeles, CA",105857,"[19639, 14343, 9136, 2259, 5666, 24512, 4957, ...","[5.0, 10.0, 25.0, 30.0, 35.0, 50.0, 75.0, 100....","Bring Reading Rainbow Back for Every Child, Ev...",/projects/readingrainbow/bring-reading-rainbow...
3,5702153.0,\nUPDATED: This is it. We're making a Veronica...,Rob Thomas,Narrative Film,usd,2000000.0,"San Diego, CA",91585,"[5938, 8423, 11509, 22997, 23227, 1865, 7260, ...","[1.0, 10.0, 25.0, 35.0, 50.0, 75.0, 100.0, 150...",The Veronica Mars Movie Project,/projects/559914737/the-veronica-mars-movie-pr...
4,3336371.0,"\nAn adventure game from Tim Schafer, Double F...",Double Fine and 2 Player Productions,Video Games,usd,400000.0,"San Francisco, CA",87142,"[47946, 24636, 1090, 11530, 900, 148, 100, 10, 4]","[15.0, 30.0, 60.0, 100.0, 250.0, 500.0, 1000.0...",Double Fine Adventure,/projects/doublefine/double-fine-adventure


In [17]:
back.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 12 columns):
amt.pledged         4000 non-null float64
blurb               4000 non-null object
by                  4000 non-null object
category            4000 non-null object
currency            4000 non-null object
goal                4000 non-null float64
location            4000 non-null object
num.backers         4000 non-null int64
num.backers.tier    4000 non-null object
pledge.tier         4000 non-null object
title               4000 non-null object
url                 4000 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 406.2+ KB


In [14]:
back.describe()

,amt.pledged,goal,num.backers
count,4.000000e+03,4.000000e+03,4000.000000
mean,2.899923e+05,6.175203e+04,3582.875750
std,7.119369e+05,1.266715e+05,7316.102809
min,1.925000e+03,1.000000e+00,1109.000000
25%,6.771300e+04,1.300000e+04,1406.750000
50%,1.205860e+05,3.000000e+04,1946.000000
75%,2.593680e+05,6.000000e+04,3372.000000
max,2.033899e+07,2.000000e+06,219382.000000
